In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
import glob
files = glob.glob("../../datasets/camel-1.4.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [25]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,camel,1.4,org.apache.camel.view.RouteDotGenerator,8,2,0,8,31,28,1,...,0.000000,0,0.588235,0.347222,1,1,48.250000,7,3.1250,1
1,camel,1.4,org.apache.camel.component.cxf.feature.Message...,5,0,0,9,17,10,2,...,0.000000,0,0.000000,0.450000,0,0,18.400000,2,1.0000,0
2,camel,1.4,org.apache.camel.builder.script.Groovy,1,1,0,0,1,0,0,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
3,camel,1.4,org.apache.camel.component.jhc.JhcProducer,13,3,0,41,73,50,5,...,0.714286,0,0.645161,0.250000,1,1,24.923077,5,1.6154,0
4,camel,1.4,org.apache.camel.bam.model.ActivityDefinition,7,2,0,6,13,17,4,...,1.000000,1,0.333333,0.428571,0,0,5.285714,1,0.8571,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,camel,1.4,org.apache.camel.component.mail.MailConfiguration,47,1,0,7,81,829,5,...,0.857143,0,0.000000,0.183511,0,0,13.446809,9,1.4468,0
868,camel,1.4,org.apache.camel.util.ExpressionListComparator,3,1,0,4,9,1,1,...,1.000000,0,0.000000,0.500000,0,0,14.333333,2,1.0000,0
869,camel,1.4,org.apache.camel.model.language.PhpExpression,3,2,0,1,5,3,0,...,0.000000,0,0.956522,0.666667,1,2,3.000000,1,0.3333,0
870,camel,1.4,org.apache.camel.util.IntrospectionSupport,20,1,0,18,86,188,14,...,1.000000,0,0.000000,0.276316,0,0,35.900000,11,2.5000,1


In [26]:
train, test = train_test_split(ant_master, test_size=0.2)

In [27]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [28]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
836,5,1,0,19,54,4,1,18,3,0.250000,274,1.0,0,0.500000,0.500000,0,0,53.600000,1,0.6000
704,4,4,8,12,10,0,10,2,3,0.833333,39,1.0,1,0.894737,0.625000,0,0,8.250000,1,0.5000
151,15,3,0,5,22,71,1,5,13,0.785714,113,1.0,0,0.729167,0.350000,2,6,6.333333,2,0.9333
719,10,4,0,6,27,5,1,5,8,0.527778,168,1.0,2,0.740741,0.250000,2,2,15.400000,3,1.1000
252,0,1,0,0,0,0,0,0,0,2.000000,0,0.0,0,0.000000,0.000000,0,0,0.000000,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,31,1,0,24,52,459,11,23,31,0.900000,165,1.0,3,0.000000,0.148387,0,0,4.225806,1,0.9677
526,18,1,0,20,18,153,12,17,18,2.000000,18,0.0,0,0.000000,0.238095,0,0,0.000000,1,1.0000
200,1,1,0,27,1,0,24,3,1,2.000000,1,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000
814,4,3,0,6,8,0,0,6,3,0.333333,27,1.0,0,0.925000,0.625000,0,0,5.500000,1,0.7500


In [29]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
824,21,2,1,15,52,142,1,14,12,0.828571,292,1.0,1,0.333333,0.200000,1,2,12.571429,3,1.0000
211,12,1,3,8,25,46,6,7,8,0.454545,102,0.0,1,0.000000,0.226190,0,0,7.416667,2,1.1667
511,4,3,0,4,7,0,1,4,3,0.333333,28,1.0,0,0.918919,0.437500,1,1,5.750000,1,0.7500
236,2,1,0,17,2,1,16,1,2,2.000000,2,0.0,0,0.000000,0.700000,0,0,0.000000,1,1.0000
365,18,1,0,1,34,149,1,0,18,0.558824,114,1.0,0,0.000000,0.183333,0,0,5.222222,2,1.0556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,8,2,0,14,17,0,8,7,6,0.571429,69,1.0,1,0.650000,0.300000,1,1,7.375000,2,1.0000
90,1,1,0,6,1,0,6,0,1,2.000000,1,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000
186,2,1,0,3,2,1,2,1,2,2.000000,2,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000
68,3,1,0,4,5,3,0,4,3,2.000000,18,0.0,0,0.000000,0.833333,0,0,5.000000,1,0.6667


In [30]:
y_train = train['bug']
y_test = test['bug']

In [31]:
y_train

836    0
704    0
151    0
719    0
252    0
      ..
256    0
526    0
200    0
814    0
486    0
Name: bug, Length: 697, dtype: int64

In [32]:
y_test

824    0
211    0
511    0
236    0
365    0
      ..
275    0
90     0
186    0
68     0
460    0
Name: bug, Length: 175, dtype: int64

In [33]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [34]:
x_train_np

array([[ 5.        ,  1.        ,  0.        , ..., 53.6       ,
         1.        ,  0.6       ],
       [ 4.        ,  4.        ,  8.        , ...,  8.25      ,
         1.        ,  0.5       ],
       [15.        ,  3.        ,  0.        , ...,  6.33333333,
         2.        ,  0.9333    ],
       ...,
       [ 1.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 4.        ,  3.        ,  0.        , ...,  5.5       ,
         1.        ,  0.75      ],
       [ 2.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [35]:
x_test_np

array([[21.        ,  2.        ,  1.        , ..., 12.57142857,
         3.        ,  1.        ],
       [12.        ,  1.        ,  3.        , ...,  7.41666667,
         2.        ,  1.1667    ],
       [ 4.        ,  3.        ,  0.        , ...,  5.75      ,
         1.        ,  0.75      ],
       ...,
       [ 2.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 3.        ,  1.        ,  0.        , ...,  5.        ,
         1.        ,  0.6667    ],
       [ 7.        ,  2.        ,  0.        , ..., 12.42857143,
         2.        ,  1.        ]])

In [36]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [37]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [38]:
train_x

array([[[ 5.        ,  1.        ,  0.        , ..., 53.6       ,
          1.        ,  0.6       ]],

       [[ 4.        ,  4.        ,  8.        , ...,  8.25      ,
          1.        ,  0.5       ]],

       [[15.        ,  3.        ,  0.        , ...,  6.33333333,
          2.        ,  0.9333    ]],

       ...,

       [[ 1.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]],

       [[ 4.        ,  3.        ,  0.        , ...,  5.5       ,
          1.        ,  0.75      ]],

       [[ 2.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]]])

In [39]:
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 2, 0, 1, 0,
       0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 5,
       0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 6, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [40]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [41]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
22/22 [==============================] - 4s 12ms/step - loss: 1.9129 - mse: 1.9129 - mae: 0.4245 - root_mean_squared_error: 1.3831 - mean_squared_logarithmic_error: 0.2240
Epoch 2/100
22/22 [==============================] - 0s 5ms/step - loss: 1.8090 - mse: 1.8090 - mae: 0.5513 - root_mean_squared_error: 1.3450 - mean_squared_logarithmic_error: 0.2062
Epoch 3/100
22/22 [==============================] - 0s 5ms/step - loss: 1.7135 - mse: 1.7135 - mae: 0.6238 - root_mean_squared_error: 1.3090 - mean_squared_logarithmic_error: 0.2089
Epoch 4/100
22/22 [==============================] - 0s 5ms/step - loss: 1.6701 - mse: 1.6701 - mae: 0.5734 - root_mean_squared_error: 1.2923 - mean_squared_logarithmic_error: 0.1953
Epoch 5/100
22/22 [==============================] - 0s 5ms/step - loss: 1.6108 - mse: 1.6108 - mae: 0.5911 - root_mean_squared_error: 1.2692 - mean_squared_logarithmic_error: 0.1980
Epoch 6/100
22/22 [==============================] - 0s 5ms/step - loss: 1.5820 - ms

22/22 [==============================] - 0s 7ms/step - loss: 0.9315 - mse: 0.9315 - mae: 0.4421 - root_mean_squared_error: 0.9651 - mean_squared_logarithmic_error: 0.1381
Epoch 45/100
22/22 [==============================] - 0s 5ms/step - loss: 1.0649 - mse: 1.0649 - mae: 0.4566 - root_mean_squared_error: 1.0319 - mean_squared_logarithmic_error: 0.1448
Epoch 46/100
22/22 [==============================] - 0s 5ms/step - loss: 1.0062 - mse: 1.0062 - mae: 0.4444 - root_mean_squared_error: 1.0031 - mean_squared_logarithmic_error: 0.1350
Epoch 47/100
22/22 [==============================] - 0s 7ms/step - loss: 0.9124 - mse: 0.9124 - mae: 0.4317 - root_mean_squared_error: 0.9552 - mean_squared_logarithmic_error: 0.1224
Epoch 48/100
22/22 [==============================] - 0s 5ms/step - loss: 0.8542 - mse: 0.8542 - mae: 0.4077 - root_mean_squared_error: 0.9242 - mean_squared_logarithmic_error: 0.1251
Epoch 49/100
22/22 [==============================] - 0s 5ms/step - loss: 0.9335 - mse: 0.933

22/22 [==============================] - 0s 6ms/step - loss: 0.6107 - mse: 0.6107 - mae: 0.3245 - root_mean_squared_error: 0.7815 - mean_squared_logarithmic_error: 0.0915
Epoch 89/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5859 - mse: 0.5859 - mae: 0.3404 - root_mean_squared_error: 0.7655 - mean_squared_logarithmic_error: 0.0906
Epoch 90/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5757 - mse: 0.5757 - mae: 0.3153 - root_mean_squared_error: 0.7588 - mean_squared_logarithmic_error: 0.0889
Epoch 91/100
22/22 [==============================] - 0s 6ms/step - loss: 0.5561 - mse: 0.5561 - mae: 0.3173 - root_mean_squared_error: 0.7458 - mean_squared_logarithmic_error: 0.0886
Epoch 92/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5745 - mse: 0.5745 - mae: 0.3252 - root_mean_squared_error: 0.7580 - mean_squared_logarithmic_error: 0.0925
Epoch 93/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5759 - mse: 0.575

In [42]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:32:19 ago. (Use '!kill 3750' to kill it.)

In [43]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 1s 2ms/step - loss: 1.5963 - mse: 1.5963 - mae: 0.5182 - root_mean_squared_error: 1.2634 - mean_squared_logarithmic_error: 0.2173


[1.596260666847229,
 1.596260666847229,
 0.5182356238365173,
 1.2634321451187134,
 0.21731001138687134]